# Modelos Lineales Mixtos en R

## Índice
1. [Introducción](#intro)
2. [Configuración del Entorno](#setup)
3. [Conceptos Fundamentales](#concepts)
4. [Análisis con Distribución Normal](#normal)
5. [Análisis con Distribución de Poisson](#poisson)
6. [Análisis con Distribución Binomial Negativa](#negative-binomial)
7. [Análisis con Distribución Cero-Inflada](#zero-inflated)
8. [Pruebas Post Hoc y Efectos Marginales](#post-hoc)
9. [Conclusiones](#conclusions)
10. [Referencias](#references)

## 1. Introducción <a name="intro"></a>

Los modelos lineales mixtos son herramientas estadísticas poderosas para analizar datos que presentan estructuras complejas, como medidas repetidas o datos jerárquicos. En este documento, exploraremos cómo implementar y interpretar estos modelos en R, utilizando diferentes distribuciones de datos: normal, Poisson, binomial negativa y cero-inflada.

## 2. Configuración del Entorno <a name="setup"></a>

Primero, instalamos y cargamos las librerías necesarias:

In [2]:
# Crear un directorio en tu espacio de usuario para los paquetes de R
dir.create('~/R/x86_64-pc-linux-gnu-library/4.0', recursive = TRUE)

# Instalar los paquetes en ese directorio
install.packages(c("nlme", "glmmTMB", "emmeans", "multcomp", "MASS", "ggplot2", "patchwork"), lib = '~/R/x86_64-pc-linux-gnu-library/4.0')
install.packages('lme4', repos='http://cran.rstudio.com/', lib = '~/R/x86_64-pc-linux-gnu-library/4.0')
install.packages("TMB", type = "source", lib = '~/R/x86_64-pc-linux-gnu-library/4.0')

# Asegurarse de que R use el directorio local para los paquetes
.libPaths('~/R/x86_64-pc-linux-gnu-library/4.0')



also installing the dependencies ‘rprojroot’, ‘rstudioapi’, ‘diffobj’, ‘rematch2’, ‘brio’, ‘callr’, ‘desc’, ‘pkgload’, ‘praise’, ‘processx’, ‘ps’, ‘waldo’, ‘testthat’, ‘colorspace’, ‘minqa’, ‘nloptr’, ‘Rcpp’, ‘zoo’, ‘farver’, ‘labeling’, ‘munsell’, ‘R6’, ‘RColorBrewer’, ‘viridisLite’, ‘ellipsis’, ‘magrittr’, ‘pkgconfig’, ‘TMB’, ‘lme4’, ‘numDeriv’, ‘RcppEigen’, ‘estimability’, ‘mvtnorm’, ‘xtable’, ‘TH.data’, ‘sandwich’, ‘gtable’, ‘isoband’, ‘scales’, ‘tibble’, ‘withr’


also installing the dependency ‘RcppEigen’




In [3]:


# Carga de librerías
.libPaths(c("/home/docker/R/x86_64-pc-linux-gnu-library/4.1", .libPaths()))
library(lme4)        # Modelos lineales mixtos
#lme4::lmList
library(nlme)        # Modelos lineales y no lineales mixtos
library(glmmTMB)     # Modelos mixtos con distribuciones avanzadas
library(emmeans)     # Estimación de medias marginales
library(multcomp)    # Comparaciones múltiples
library(MASS)        # Funciones estadísticas adicionales
#MASS::geyser
library(ggplot2)     # Visualización de datos
library(patchwork)   # Combinación de gráficos
#patchwork::area

Loading required package: Matrix


Attaching package: ‘nlme’


The following object is masked from ‘package:lme4’:

    lmList


Warning message in checkMatrixPackageVersion():
“Package version inconsistency detected.
TMB was built with Matrix version 1.4.0
Current Matrix version is 1.3.4
Please re-install 'TMB' from source using install.packages('TMB', type = 'source') or ask CRAN for a binary version of 'TMB' matching CRAN's 'Matrix' package”
Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser



Attaching package: ‘patchwork’


The following object is masked from ‘package:MASS’:

    area




## 3. Conceptos Fundamentales <a name="concepts"></a>

> **Nota:** Los modelos lineales mixtos (MLM) combinan efectos fijos y aleatorios para capturar tanto las relaciones sistemáticas como la variabilidad no explicada en los datos.

La elección de la distribución adecuada es crucial para modelar correctamente los datos, incluyendo distribuciones normal, Poisson, binomial negativa y cero-inflada.

## 4. Análisis con Distribución Normal <a name="normal"></a>

En este análisis, ajustamos un modelo lineal mixto utilizando la distribución normal para modelar variables continuas. Por ejemplo, podemos estudiar cómo un programa educativo afecta los puntajes de los estudiantes.

In [4]:
# Ajuste del modelo
model_normal <- lmer(score ~ program + (1 | school), data = data_normal)

# Resumen del modelo
summary(model_normal)

ERROR: Error: bad 'data': object 'data_normal' not found


Interpretación: El coeficiente asociado al programa indica si hubo un efecto significativo del programa experimental en comparación con el grupo de control.

## 5. Análisis con Distribución de Poisson <a name="poisson"></a>

La distribución de Poisson se utiliza para modelar conteos de eventos. Aquí se muestra cómo ajustar un modelo utilizando esta distribución para analizar, por ejemplo, el número de defectos en una línea de producción.

In [ ]:
# Ajuste del modelo de Poisson
model_poisson <- glmer(defects ~ hours + (1 | machine), family = poisson, data = data_poisson)

# Resumen del modelo
summary(model_poisson)

Interpretación: El modelo nos permite evaluar cómo las horas de operación afectan el número de defectos observados.

## 6. Análisis con Distribución Binomial Negativa <a name="negative-binomial"></a>

La distribución binomial negativa es útil cuando existe sobredispersión en los datos de conteo. Ajustamos un modelo binomial negativo para mejorar el ajuste respecto a un modelo de Poisson.

In [ ]:
# Ajuste del modelo binomial negativo
model_negbin <- glmer.nb(defects ~ hours + (1 | machine), data = data_poisson)

# Resumen del modelo
summary(model_negbin)

Interpretación: Este modelo permite manejar mejor la variabilidad adicional presente en los datos de conteo.

## 7. Análisis con Distribución Cero-Inflada <a name="zero-inflated"></a>

En algunos casos, los datos de conteo contienen un exceso de ceros. La distribución cero-inflada nos permite modelar estos datos de manera más precisa. Por ejemplo, el número de visitas a un sitio web donde muchos usuarios no visitan el sitio.

In [ ]:
# Ajuste del modelo cero-inflado
model_zero_inflated <- glmmTMB(visits ~ 1, ziformula = ~1, family = poisson, data = data_zero_inflated)

# Resumen del modelo
summary(model_zero_inflated)

Interpretación: El modelo incluye un componente para el exceso de ceros, permitiendo una mejor representación de los datos.

## 8. Pruebas Post Hoc y Efectos Marginales <a name="post-hoc"></a>

Las pruebas post hoc y los efectos marginales son útiles para interpretar los resultados de los modelos y realizar comparaciones entre grupos. Utilizamos la librería `emmeans` para estimar medias marginales y realizar comparaciones múltiples.

In [ ]:
# Medias marginales para el modelo normal
emmeans_normal <- emmeans(model_normal, ~ program)

# Comparaciones múltiples
contrast(emmeans_normal, method = "pairwise", adjust = "tukey")

Interpretación: Estas pruebas nos ayudan a entender mejor las diferencias entre los grupos después del ajuste del modelo.

## 9. Conclusiones <a name="conclusions"></a>

En este documento, hemos explorado cómo aplicar modelos lineales mixtos en R utilizando diferentes distribuciones de datos. Cada distribución tiene sus propias características y es importante seleccionar la adecuada para modelar correctamente los datos.

Los modelos lineales mixtos nos permiten capturar tanto efectos fijos como aleatorios, proporcionando una comprensión más completa de los datos jerárquicos o con medidas repetidas.

## 10. Referencias <a name="references"></a>

- Pinheiro, J. C., & Bates, D. M. (2000). *Mixed-Effects Models in S and S-PLUS*. Springer.
- Gelman, A., & Hill, J. (2007). *Data Analysis Using Regression and Multilevel/Hierarchical Models*. Cambridge University Press.
- [Librería lme4](https://cran.r-project.org/web/packages/lme4/index.html)
- [Librería glmmTMB](https://cran.r-project.org/web/packages/glmmTMB/index.html)
- [Vignettes de emmeans](https://cran.r-project.org/web/packages/emmeans/vignettes/emmeans.html)